In [65]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from afutil import get_patch_metadata, read_or_calc_focal_planes, compile_deterministic_data,\
    feature_vector_generator_fn, plot_results, MagellanWithAnnotation
from defocusnetwork import DefocusNetwork


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Explanation of whats happening here

In [66]:
class DataWrapper:
    """
    This class wraps all the essential functionality needed for training single shot autofocus. By default, it reads
    Micro-Magellan datasets (with an added subclass to be able to store the results of intermediate computations in 
    an HDF5 file), but this could be replaced with any data source so long as it provides the following
    methods and fields
    """
    def __init__(self, magellan):
        self.magellan = magellan

    #TODO: remember to change to float
    def read_ground_truth_image(self, position_index, z_index):
        """
        Read image in which focus quality can be measured form quality of image
        :param pos_index: index of xy position
        :param z_index: index of z slice (starting at 0)
        :param xy_slice: (cropped region of image)
        :return:
        """
        return self.magellan.read_image(channel_name='DPC_Bottom', pos_index=position_index, 
                                        z_index=z_index).astype(np.float)

    def read_prediction_image(self, position_index, z_index, patch_index, split_k):
        """
        Read image used for single shot prediction (i.e. single LED image)
        :param pos_index: index of xy position
        :param z_index: index of z slice (starting at 0)
        :param split_k: number of crops along each dimension
        :param patch_index: index of the crop
        :return:
        """
        patch_size, patches_per_image = get_patch_metadata((self.get_image_width(),
                                                            self.get_image_height()), split_k)
        y_tile_index = patch_index // split_k
        x_tile_index = patch_index % split_k
        xy_slice = [[y_tile_index * patch_size, (y_tile_index + 1) * patch_size],
                    [x_tile_index * patch_size, (x_tile_index + 1) * patch_size]]
        return self.magellan.read_image(channel_name='autofocus', pos_index=position_index,
                                   z_index=z_index)[xy_slice[0][0]:xy_slice[0][1]][xy_slice[1][0]:xy_slice[1][1]]
        
    def get_image_width(self):
        """
        :return: image width in pixels
        """
        return self.magellan.image_width

    def get_image_height(self):
        """
        :return: image height in pixels
        """
        return self.magellan.image_height

    def get_num_z_slices_at(self, position_index):
        """
        return number of z slices (i.e. focal planes) at the given XY position
        :param position_index:
        :return:
        """
        return self.magellan.get_num_z_slices_at(position_index)

    def get_pixel_size_z_um(self):
        """
        :return: distance in um between consecutive z slices
        """
        return self.magellan.pixel_size_z_um

    def get_num_xy_positions(self):
        """
        :return: total number of xy positons in data set
        """
        return self.magellan.get_num_xy_positions()

    def store_focal_plane(self, name, focal_position):
        """
        Store the computed focal plane as a string, float pair
        """
        self.magellan.write_annotation(name, focal_position)

    def read_focal_plane(self, name):
        """
        read a previously computed focal plane
        :param name: key corresponding to an xy position for whch focal plane has already been computed
        :return:
        """
        return self.magellan.read_annotation(name)

    def store_array(self, name, array):
        """
        Store a numpy array containing the design matrix for training the non-deterministic part of the network (i.e.
        after the Fourier transform) so that it can be retrained quickly without having to recompute
        :param name:
        :param array: (n examples) x (d feature length) numpy array
        """
        self.magellan.store_array(name, array)

    def read_array(self, name):
        """
        Read and return a previously computed array
        :param name:
        :return:
        """
        return self.magellan.read_array(name)

First, load data and compute the ground truth focal planes as targets for training. The show_output flag will create a plot of the averaged high frequency content of the log power spectrum. The maximum of the this plot should correspond to the correct focal plane

In [67]:
#parameters for the deterministic part of the network
#TODO: better explain what these mean
deterministic_params = {'non_led_width': 0.1, 'led_width': 0.6, 'tile_split_k': 2}

#load data
data = [DataWrapper(MagellanWithAnnotation(
    '/home/henry/data/2018-9-27 Cells and histology af data/unstained path section 12x12 30um range 1um step_1'))]

#load or compute target focal planes using 22 CPU cores to speed computation
train_focal_planes = {dataset: read_or_calc_focal_planes(dataset, split_k=deterministic_params['tile_split_k'],
                                                         n_cores=6, show_output=True) for dataset in data}

Reading precomputed focal plane
Reading precomputed focal plane
Reading precomputed focal plane
Reading precomputed focal plane
Reading precomputed focal plane
Reading precomputed focal plane
Reading precomputed focal plane
Reading precomputed focal plane
Reading precomputed focal plane
Reading precomputed focal plane
Reading precomputed focal plane
Reading precomputed focal plane
Reading precomputed focal plane
Reading precomputed focal plane
Reading precomputed focal plane
Reading precomputed focal plane
Reading precomputed focal plane
Reading precomputed focal plane
Reading precomputed focal plane
Reading precomputed focal plane
Reading precomputed focal plane
Reading precomputed focal plane
Reading precomputed focal plane
Reading precomputed focal plane
Reading precomputed focal plane
Reading precomputed focal plane
Reading precomputed focal plane
Reading precomputed focal plane
Reading precomputed focal plane
Reading precomputed focal plane
Reading precomputed focal plane
Reading 

LEDs in vertical axis of array:
4 12 28 48 83 119 187
3 11 27 47 84 120 188
3 has defect in it, 4 doesnt

In [70]:
#compute or load already computed design matrices
features, targets = compile_deterministic_data(data, train_focal_planes, deterministic_params=deterministic_params)

#make genenrator function for providing training examples and seperate validation generator for assessing its progress
train_generator = feature_vector_generator_fn(features, targets, mode='training', 
                                        split_k=deterministic_params['tile_split_k'], training_fraction=0.8)
val_generator = feature_vector_generator_fn(features, targets, mode='validation', 
                                        split_k=deterministic_params['tile_split_k'], training_fraction=0.8)

#feed in the dimensions of the cropped input so the inference network knows what to expect
patch_size, patches_per_image = get_patch_metadata((data[0].get_image_width(),
                                        data[0].get_image_height()), deterministic_params['tile_split_k'])

#this call creates network and trains it
defocus_prediction_network = DefocusNetwork(input_shape=features.shape[1], train_generator=train_generator,
                             val_generator=val_generator, predict_input_shape=[patch_size, patch_size],
                             deterministic_params=deterministic_params, regressor_only=True, train_mode='train')

2961 sliceposition tuples


Building trainable graph...


AttributeError: 'DefocusNetwork' object has no attribute 'hyperparams'

Analyze performance and stuff

In [6]:
#visualize results
#validation data drawn from same set used in training, test data is a seperate file
train_prediction_defocus, train_target_defocus = defocus_prediction_network.analyze_performance(
    feature_vector_generator_fn(train_features, train_targets, mode='all', split_k=deterministic_params['tile_split_k']))

test_prediction_defocus, test_target_defocus = defocus_prediction_network.analyze_performance(
        feature_vector_generator_fn(test_features, test_targets, mode='all', split_k=deterministic_params['tile_split_k']))

#average predictions
train_pred_avg, train_target_avg = average_predictions(train_prediction_defocus, train_target_defocus, deterministic_params['tile_split_k'] ** 2)
test_pred_avg, test_target_avg = average_predictions(test_prediction_defocus, test_target_defocus, deterministic_params['tile_split_k']**2)

plt.figure(1)
plot_results(train_prediction_defocus, train_target_defocus, 'Training')
plot_results(test_prediction_defocus, test_target_defocus, 'Test', draw_rect=True)
plt.legend(['Training data average', 'Test data averaged', 'Ground truth', 'Objective depth of focus'])

#show data before tile averaging
plt.figure(2)
plot_results(train_pred_avg, train_target_avg, 'Training')
plot_results(test_pred_avg, test_target_avg, 'Test', draw_rect=True)
plt.legend(['Training data average', 'Test data averaged', 'Ground truth', 'Objective depth of focus'])

plt.show(block=True)

NameError: name 'defocus_prediction_network' is not defined